In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug  8 16:48:46 2020

@author: user1
"""
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener 
from tweepy import OAuthHandler
import json
from pymongo import MongoClient
import pandas as pd
import sys

class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def on_connect(self):
        """Called when the connection is made"""
        print("You're connected to the streaming server.")

    def on_error(self, status_code):
        """This is called when an error occurs"""
        print('Error: ' + repr(status_code))
        return False

    def on_data(self, data):
        """This will be called each time we receive stream data"""
        client = MongoClient()

        # I stored the tweet data in a database called 'training_tweets' in MongoDB, if 
        # 'training_tweets' does not exist it will be created for you.
        db = client.bengaluru15

        # Decode JSON
        datajson = json.loads(data)

        # I'm only storing tweets in English. I stored the data for these tweets in a collection 
        # called 'training_tweets_collection' of the 'training_tweets' database.
        if "lang" in datajson and datajson["lang"] == "en":
            db.bengaluru_col.insert_one(datajson)


if __name__ == "__main__":

    # These are provided to you through the Twitter API after you create a account
    consumer_key = "oXlMIIvJaNXOSOXCMAGW6kpkl"
    consumer_secret = "4bYVRKeEViTt7fwzrxwEKS26UVKVQotRgCavx0FF9cWVfhvHhI"
    access_token = "1249305586660823040-aoy6uKUYGkTrLZMIPgVKoQbjtAXkrs"
    access_token_secret = "659B2jWYzz4niFGlMT0YXWqoDB5WUCjBfUnkMCmFWvX4Z"

    auth1 = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth1.set_access_token(access_token, access_token_secret,)


    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the 
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box. 
    LOCATIONS = [77.401137, 12.776971, 77.78503, 13.19671] 

    stream_listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
    stream = tweepy.Stream(auth=auth1, listener=stream_listener)
    stream.filter(locations=LOCATIONS)

You're connected to the streaming server.
